# Import

In [ ]:
!git clone -b deep-learning-models https://github.com/MohamadElnomrossie/Omdena-seniment-analysis/

fatal: destination path 'Omdena-seniment-analysis' already exists and is not an empty directory.


In [ ]:
%cd /content/Omdena-seniment-analysis/

/content/Omdena-seniment-analysis


In [ ]:
!pip install -U tensorflow-addons

Requirement already up-to-date: tensorflow-addons in /usr/local/lib/python3.7/dist-packages (0.13.0)


# Main

In [ ]:
import os
import json

import numpy as np
import pandas as pd

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from sklearn.model_selection import StratifiedKFold
import tensorflow_addons as tfa
import tensorflow as tf


from Sentiment import SentimentAnalysis
from utils import helper, preprocess
# from utils.config import config

### Analysis

In [ ]:
import pandas as pd
data = pd.read_csv("Datasets/Final_Dataset/Dataset/train.csv")
print(data.columns)
data.head(10)

Index(['Unnamed: 0', 'Text', 'Class_camel', 'cleaned_text', 'mentions',
       'word_count', 'contain_emoji', 'demoji_text', 'URLS'],
      dtype='object')


,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]
5,12,إذا كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، ...,negative,اذا كانت مساله حقوق المراه امرا سخيفا جدا فالل...,[],18,False,NaN,[]
6,13,@naifco كل هذه الحقوق او المزعوم انها حقوق عبا...,negative,كل هذه الحقوق او المزعوم انها حقوق عباره عن صف...,['naifco'],16,False,NaN,[]
7,14,متى تعرف ان المرأة ما عندها حقوق؟\n يوم يطبعون...,neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH']
8,16,RT @miss_wahaj: @RmNxb المسيار زي زواج المتعه ...,negative,المسيار زي زواج المتعه عند الشيعه نعايب عليهم ...,"['miss', 'RmNxb']",13,False,NaN,[]
9,18,RT @Gh08944301: @Eqbal_Darandari لمتى واحنا بل...,negative,لمتي واحنا بلا حقوق لانقدر نجدد جوازات سفرنا و...,"['Gh08944301', 'Eqbal']",50,False,NaN,[]


In [ ]:
print(data.cleaned_text.loc[45], data.Class_camel.loc[45])
print(data.cleaned_text.loc[0], data.Class_camel.loc[0])
print(data.cleaned_text.loc[35], data.Class_camel.loc[35])

لان تبني حقوق المراه من وجهه نظر علمانيه وتحرريه يضر بمجتمعي ويضر بالنساء والرجال علي حد سواء بس هم ليش مهتمين وحاشرين خشومهم negative
حقوق المراه neutral
تضامن مع ترف العسيري اختي المسلمه حجابك حشمتك حيائك هي هويتك الحقيقيه التي كرمك بها دينك العظيم حافظي عليها فهي من تمنحك الكرامه والحريه الحقيقيه لاتستمعي لمن يحاول طمس هويتك الاسلاميه تحت دعاوي مزيفه مثل حقوق المراه وبكائيات من تسمين انفسهن ب الناشطات والحقوقيات positive


In [ ]:
data.Class_camel.value_counts()

positive    57096
negative    33702
neutral     20124
Name: Class_camel, dtype: int64

### Config

In [ ]:
import pickle
with open("Datasets/stopWords.pkl", 'rb') as f:
    stop_words = list(pickle.load(f))
    stop_words = list(set(stop_words + ['و','في','من','بواسطة','أ','هو','و','في','سيكون','إلى','كان','كن','هو','ال','و','ما','ء','ه','س']))

config = {
    'vocab_size':60000,
    'maxlen':180,
    'embedding_vector':300,

    'method':'lstm', #other - simpleRNN, bidRNN, 1DConv, lstm
    'stop_words':stop_words,
    'punctuations':"""'!"-#$%&'()*+,«».؛،/:؟?@[\]^_`{|}~""",

    'epochs':120,
    'batch_size':64,
    'optim':'SGD', # other - adamax, adadelta, SGD, Adam, RMSprop
    'learning_rate':1e-2,

    'save_model_path':'models/',
    'save_weights_path':"models/",
    'train_data_path':"Datasets/Final_Dataset/Dataset/train.csv",
    'val_data_path':"Datasets/Final_Dataset/Dataset/val.csv",
    'test_data_path':"Datasets/Final_Dataset/Dataset/test.csv",
}


### Train

In [ ]:
train_data = pd.read_csv(config['train_data_path'])
val_data = pd.read_csv(config['val_data_path'])
test_data = pd.read_csv(config['test_data_path'])

train_data = train_data.dropna().reset_index(drop=True)
val_data = val_data.dropna().reset_index(drop=True)
test_data = test_data.dropna().reset_index(drop=True)

train_data = train_data.sample(frac=1).reset_index(drop=True)
val_data = val_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

train_text, train_label = train_data['cleaned_text'].values.copy(), train_data['Class_camel'].values.copy()
val_text, val_label = val_data['cleaned_text'].values.copy(), val_data['Class_camel'].values.copy()
test_text, test_label = test_data['cleaned_text'].values.copy(), test_data['Class_camel'].values.copy()


In [ ]:
#Default Tokenization
sentiment = SentimentAnalysis(preprocess.tokenizer, vocab_size=config['vocab_size'], maxlen=config['maxlen'], embedding_vector=config['embedding_vector'], method=config['method'],)

train_text = sentiment.tokenize(train_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
train_text, train_label, unique_words, word_dict = sentiment.vectorize(train_text, train_label, return_label=True)

val_text = sentiment.tokenize(val_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
val_text, val_label, _, _ = sentiment.vectorize(val_text, val_label, return_label=True)

test_text = sentiment.tokenize(test_text, punctuations=config['punctuations'], stop_words=config['stop_words'])
test_text, test_label, _, _ = sentiment.vectorize(test_text, test_label, return_label=True)


In [ ]:
train_data.cleaned_text.iloc[0], train_data.Class_camel.iloc[0] 

('معروفه لاتخربون علي', 'negative')

In [ ]:
train_text[0], train_label[0]

(array([17354, 59414, 54006,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [ ]:
model = sentiment.fit(train_text,
                    train_label,
                    validation_data=(test_text, test_label),
                    epochs=config['epochs'],
                    batch_size=config['batch_size'],
                    method=config['method'])
sentiment.evaluate(test_text, test_label, model, batch_size=16)

Epoch 1/120



Epoch 00001: val_loss improved from inf to 0.99305, saving model to models/lstm_model.h5

Epoch 2/120



Epoch 00002: val_loss improved from 0.99305 to 0.99137, saving model to models/lstm_model.h5

Epoch 3/120



Epoch 00003: val_loss improved from 0.99137 to 0.98928, saving model to models/lstm_model.h5

Epoch 4/120



Epoch 00004: val_loss improved from 0.98928 to 0.98688, saving model to models/lstm_model.h5

Epoch 5/120


...


Epoch 00116: val_loss did not improve from 0.58952

Epoch 117/120



Epoch 00117: val_loss did not improve from 0.58952

Epoch 118/120



Epoch 00118: val_loss did not improve from 0.58952

Epoch 119/120



Epoch 00119: val_loss did not improve from 0.58952

Epoch 120/120



Epoch 00120: val_loss did not improve from 0.58952


978/978 [==============================] - 10s 11ms/step - loss: 1.1123 - accuracy: 0.7774 - precision_1: 0.7833 - recall_1: 0.7724 - auc_1: 0.8895

Validation loss: 1.1122727394104004  Validation acc: 0.777408242225647 Precision: 0.7832911610603333 Recall: 0.7724190950393677 Auc Roc: 0.8895121216773987


### Validation

In [ ]:
print(f"Text : {test_data['cleaned_text'].iloc[45]} Label : {test_data['Class_camel'].iloc[45]}")
sentiment.predict_([test_data['cleaned_text'].iloc[45]], model, batch_size=32, print_=True)

print(f"Text : {test_data['cleaned_text'].iloc[7]} Label : {test_data['Class_camel'].iloc[7]}")
sentiment.predict_([test_data['cleaned_text'].iloc[7]], model, batch_size=32, print_=True)

print(f"Text : {test_data['cleaned_text'].iloc[120]} Label : {test_data['Class_camel'].iloc[120]}")
sentiment.predict_([test_data['cleaned_text'].iloc[120]], model, batch_size=32, print_=True)

Text : الشيء الوحيد الذي وصلوا فيه للعالميه هو المسيار تري كانوا يشجعون ريال مدريد ضد النصر Label : negative



--------------------
Negative 0.999981164932251
--------------------
Text : انا جيت عشان اسمع حواديتك حواديتي بس دي كلها عنك طبعا وده اللي مكيفني بيتر بان Label : neutral



--------------------
Neutral 0.9992473125457764
--------------------
Text : ღ اسلي مع طيفك الين انتعشتك عشان ماحس بغيابك علي طول ياجعلها في ذمتك ماوحشتك تصميمي Label : positive



--------------------
Positive 0.9999939203262329
--------------------


### Inference

In [ ]:
import pandas as pd
import tensorflow as tf
from Sentiment import SentimentAnalysis
from utils import helper, preprocess

data = pd.read_csv("Datasets/Final_Dataset/Dataset/train.csv")
data.head(10)

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1']
1,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU']
2,5,RT @fatemaar7: انا كامرأة يوجعني كل حزن نساء ا...,negative,انا كامراه يوجعني كل حزن نساء العالم سواء تعنف...,['fatemaar7'],50,False,NaN,['https://t.co/c494qBVPx4']
3,7,@AliFCD @ShamsanM @AlyemenNor @space_dist @m2r...,negative,غسق وسبيس وحلوه الحلوات هم اشخاص لهم هويتهم وك...,"['AliFCD', 'ShamsanM', 'AlyemenNor', 'space', ...",32,False,NaN,[]
4,9,RT @Dresraamohammed: السؤال بقى للناس الي شغال...,negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[]
5,12,إذا كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، ...,negative,اذا كانت مساله حقوق المراه امرا سخيفا جدا فالل...,[],18,False,NaN,[]
6,13,@naifco كل هذه الحقوق او المزعوم انها حقوق عبا...,negative,كل هذه الحقوق او المزعوم انها حقوق عباره عن صف...,['naifco'],16,False,NaN,[]
7,14,متى تعرف ان المرأة ما عندها حقوق؟\n يوم يطبعون...,neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH']
8,16,RT @miss_wahaj: @RmNxb المسيار زي زواج المتعه ...,negative,المسيار زي زواج المتعه عند الشيعه نعايب عليهم ...,"['miss', 'RmNxb']",13,False,NaN,[]
9,18,RT @Gh08944301: @Eqbal_Darandari لمتى واحنا بل...,negative,لمتي واحنا بلا حقوق لانقدر نجدد جوازات سفرنا و...,"['Gh08944301', 'Eqbal']",50,False,NaN,[]


In [ ]:
data = data.dropna().reset_index(drop=True)
text = data["Text"].values

In [ ]:
text[0:5]

array(['حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1',
       'RT @saud_talep: Retweeted لجنة التنمية بشبرا (@Shubratanmyeh):\n \n ما زال التسجيل مستمر في دورة حقوق المرأة بعد الطلاق ✨ #وعيك_يحميك... https://t.co/c2NXzNCdLU',
       'RT @Dresraamohammed: السؤال بقى للناس الي شغاله في #حقوق_الانسان #حقوق_المرأة \n يا ترى في قانون او عقوبه على الزوج الي بيمارس العنف ضد زوجته في الشارع ؟؟؟ 🤦🏻\u200d♀️👩🏻\u200d💼',
       'متى تعرف ان المرأة ما عندها حقوق؟\n يوم يطبعون لها ورقة مثل هذي 👇🏼👇🏼👇🏼 https://t.co/7a9H0KQTqH',
       '@Almajlliss اتوقع الي حاط الاستفتاء وده يخنق المرأه ويسجنها 😹😹 الحمدالله ديرتنا ديره حقوق للمرأه 🙏😋'],
      dtype=object)

In [ ]:
model = tf.keras.models.load_model("models/lstm_model.h5")
preds = sentiment.predict_(text, model, batch_size=32, print_=False)
data["prediction"] = preds

100%|██████████| 29029/29029 [00:04<00:00, 5984.11it/s]


In [ ]:
data.head()

,Unnamed: 0,Text,Class_camel,cleaned_text,mentions,word_count,contain_emoji,demoji_text,URLS,prediction
0,0,"[حقوق, المرأة, 💚💚💚, https, t, co]",neutral,حقوق المراه,[],2,True,:green_heart::green_heart::green_heart:,['https://t.co/Mzf90Ta5g1'],Negative
1,2,"[RT, saud, talep, Retweeted, لجنة, التنمية, بش...",neutral,لجنه التنميه بشبرا ما زال التسجيل مستمر في دور...,"['saud', 'Shubratanmyeh']",15,True,:sparkles:,['https://t.co/c2NXzNCdLU'],Negative
2,9,"[RT, Dresraamohammed, السؤال, بقى, للناس, الي,...",negative,السؤال بقي للناس الي شغاله في حقوق الانسان حقو...,['Dresraamohammed'],24,True,:person_facepalming_light_skin_tone::female_si...,[],Negative
3,14,"[متى, تعرف, ان, المرأة, عندها, حقوق, يوم, يطبع...",neutral,متي تعرف ان المراه ما عندها حقوق يوم يطبعون له...,[],13,True,:backhand_index_pointing_down_medium-light_ski...,['https://t.co/7a9H0KQTqH'],Negative
4,21,"[Almajlliss, اتوقع, الي, حاط, الاستفتاء, وده, ...",negative,اتوقع الي حاط الاستفتاء وده يخنق المراه ويسجنه...,['Almajlliss'],13,True,:cat_face_with_tears_of_joy::cat_face_with_tea...,[],Positive
